In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    "chest_xray/train",
    image_size=(150 , 150),
    batch_size=32,
    color_mode='grayscale',
    label_mode='binary'
)

validation_ds = tf.keras.utils.image_dataset_from_directory(
    "chest_xray/test",
    image_size=(150 , 150),
    batch_size=32,
    color_mode='grayscale',
    label_mode='binary'
)

# Augmentation Layer
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])


Found 5216 files belonging to 2 classes.
Found 624 files belonging to 2 classes.


In [3]:
model = models.Sequential([
    layers.Rescaling(1./255, input_shape=[150, 150, 1]),

    data_augmentation,

    # This is to identify the lines
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    # This is to identify the shapes
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    # This is to identify the textures
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),

    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),

    #Output
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

d:\Downloads\Necessities\Anaconda_Install\envs\ai_bot\lib\site-packages\keras\src\layers\preprocessing\data_layer.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 150, 150, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 150, 150, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     2,367,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,460,289 (9.39 MB)

 Trainable params: 2,460,289 (9.39 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'xray_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

history = model.fit(train_ds, epochs=5, validation_data=validation_ds, callbacks=[checkpoint])

Epoch 1/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step - accuracy: 0.7597 - loss: 0.4930
Epoch 1: val_accuracy improved from None to 0.83974, saving model to xray_model.h5


163/163 ━━━━━━━━━━━━━━━━━━━━ 54s 314ms/step - accuracy: 0.8204 - loss: 0.3909 - val_accuracy: 0.8397 - val_loss: 0.3435
Epoch 2/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step - accuracy: 0.8984 - loss: 0.2499
Epoch 2: val_accuracy did not improve from 0.83974
163/163 ━━━━━━━━━━━━━━━━━━━━ 51s 311ms/step - accuracy: 0.8898 - loss: 0.2698 - val_accuracy: 0.8269 - val_loss: 0.3596
Epoch 3/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step - accuracy: 0.9233 - loss: 0.1915
Epoch 3: val_accuracy improved from 0.83974 to 0.87179, saving model to xray_model.h5


163/163 ━━━━━━━━━━━━━━━━━━━━ 51s 313ms/step - accuracy: 0.9195 - loss: 0.2027 - val_accuracy: 0.8718 - val_loss: 0.3185
Epoch 4/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step - accuracy: 0.9336 - loss: 0.1600
Epoch 4: val_accuracy improved from 0.87179 to 0.88301, saving model to xray_model.h5


163/163 ━━━━━━━━━━━━━━━━━━━━ 51s 312ms/step - accuracy: 0.9358 - loss: 0.1608 - val_accuracy: 0.8830 - val_loss: 0.2883
Epoch 5/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step - accuracy: 0.9351 - loss: 0.1687
Epoch 5: val_accuracy improved from 0.88301 to 0.90545, saving model to xray_model.h5


163/163 ━━━━━━━━━━━━━━━━━━━━ 51s 315ms/step - accuracy: 0.9350 - loss: 0.1722 - val_accuracy: 0.9054 - val_loss: 0.2575
